In [2]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pickle
import re
import plotly.express as px

import os,json
from time import time
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import StratifiedKFold
import xlrd

from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from category_encoders import OneHotEncoder
from sklearn.model_selection import KFold

## Data process

In [3]:
data = pd.read_csv('data_leave_one_out.csv')
data = data.rename(columns = {'intent':'Intent','query':'Questions'})
data = data.dropna().drop_duplicates()
data['Questions'] = data['Questions'].apply(lambda x: x.lower())
data['Questions']  = data['Questions'].apply(lambda x: re.sub(r'(!|\.|,|\(|\)|\[|\]|\\|\?|\$|#|%|\*)', '', x))
data['Intent'] = data['Intent'].astype(str)

data.head(2)

,Unnamed: 0,Intent,Questions,Statement request,Passbook savings accounts,Card statements,Credit card statement,Debit card statement,Investment account statement,Home loan account statement,...,Paying a cancelled credit card,How to close my account,Card dispute,Change credit card limit,Increase credit card limit,Decrease credit card limit,Credit card application rejection,Rebates,How to redeem rewards,Update details
0,0,Statement request,like copy -pron- statement,0.239015,0.476868,0.337956,0.419239,0.569444,0.497531,0.539970,...,0.717396,0.724538,0.644826,0.740849,0.791113,0.835014,0.631769,0.770484,0.854625,0.560702
1,1,Statement request,send -pron- copy -pron- statement,0.267362,0.469997,0.365053,0.366122,0.528491,0.406568,0.507421,...,0.626802,0.666503,0.622732,0.720149,0.724000,0.784794,0.599908,0.718440,0.817008,0.487535


In [4]:
cluster_cols =['Promotions', 'Card Promotions',
       'Open Account', 'OCBC Singapore Account', 'OCBC Securities Account ',
       'OCBC Malaysia Account', 'NISP Account', 'Card Cancellation',
       'Cancel Credit or Debit Card', 'Cancel ATM Card',
       'Speak with a customer service officer', 'Request for sponsorship',
       'Repricing', 'Auto Loan', 'Home Loan', 'Service Fee',
       'Token Replacement', 'Token Activation', 'Hardware Token', 'Onetoken',
       'Late fee waiver', 'Credit card interest waiver',
       'Request for account fee waiver', 'Uplift suspension on accounts',
       'Loan Enquiry', 'Home and property loans', 'Personal Loans',
       '365 Card Application', 'Paying a cancelled credit card',
       'How to close my account', 'Credit card transaction dispute',
       'Change credit card limit', 'Increase credit card limit',
       'Decrease credit card limit', 'Credit card application rejection',
       'Rebates', 'How to redeem rewards', '360 Account interest dispute',
       'Statement Request', 'Passbook savings account statement',
       'Credit card statement', 'Debit card statement',
       'Investment account statement', 'Update details']
len(cluster_cols)

44

In [5]:
cluster_cols =[
    'Statement request',
    'Passbook savings accounts',
    'Card statements',
    'Credit card statement',
    'Debit card statement',
    'Investment account statement',
    'Home loan account statement',
    '360 Account interest dispute',
    'Change of billing cycle',
    'Token Activation',
    'Student Loan',
    'Tuition fee loan',
    'Education loan',
    'Study loan',
    'Car loan full settlement',
    'Home loan repayment',
    'Cancel Fund Transfer',
    'Cancel credit card transaction',
    'Credit Refund',
    'Account opening for foreigners',
    'Mobile Banking Issues',
    'Account Fraud',
    'Dormant Account Activation',
    'CRS Enquiries',
    'SRS Contribution',
    'Dispute status',
    'Give a compliment',
    'File a complaint',
    'Funds Transfer Status',
    'Telegraphic transfer Status',
    'Make a telegraphic transfer',
    'Unable to log into internet banking',
    'Card application status',
    'Supplementary card application',
    'Access codes for banking services',
    'Interest or Late fee waiver',
    'Annual Fee Waiver',
    'SMS Alerts',
    'Reset PIN',
    'Unsuccessful card transaction',
    'Card Renewal',
    'Card activation for overseas use',
    'Replace Card',
    'Lost or compromised cards',
    'Damaged or Faulty card',
    'Promotions',
    'Card Promotions',
    'Open Account',
    'Open OCBC Singapore Account',
    'Open OCBC Securities Account ',
    'Open OCBC Malaysia Account',
    'Open NISP Account',
    'Card Cancellation',
    'Cancel Credit or Debit Card',
    'Cancel ATM Card',
    'Speak with a customer service officer',
    'Request for sponsorship',
    'Repricing',
    'Reprice home loan',
    'Service Fee',
    'Token Replacement',
    'Request for account fee waiver',
    'Uplift suspension on accounts',
    'Loan Enquiry',
    'Card Application',
    'Apply for credit or debit cards',
    'Apply for ATM card',
    'Paying a cancelled credit card',
    'How to close my account',
    'Card dispute',
    'Change credit card limit',
    'Increase credit card limit',
    'Decrease credit card limit',
    'Credit card application rejection',
    'Rebates',
    'How to redeem rewards',
    'Update details']
len(cluster_cols)

77

### Get top3 clusters

In [6]:
data['clusters_top3'] = data.apply(lambda x: np.argsort(x[cluster_cols].values)[:3].tolist(), axis=1)

intents = cluster_cols # get all tickers
intent2index = {v: i for (i, v) in enumerate(intents)}

data['target'] = data['Intent'].apply(lambda x: intent2index[x])

top_clusters_cols = pd.DataFrame(data['clusters_top3'].values.tolist(),columns = ['clusters_1','clusters_2','clusters_3']).reset_index(drop=True)
data = data.reset_index(drop=True)
data = pd.concat([data,top_clusters_cols], axis=1)

data.drop(columns = 'clusters_top3', inplace=True)
data.drop(columns = cluster_cols, inplace=True)

In [17]:
data.head()

,Unnamed: 0,Intent,Questions,target,clusters_1,clusters_2,clusters_3,exists
0,0,Statement request,like copy -pron- statement,0,0,2,3,True
1,1,Statement request,send -pron- copy -pron- statement,0,0,2,3,True
2,2,Statement request,-pron- statement,0,2,0,3,True
3,3,Statement request,want hard copy -pron- statement,0,0,2,3,True
4,4,Statement request,statement request,0,0,2,3,True


In [7]:
# check cluster method accuracy - top 1
data[(data['clusters_1'] == data['target'])].shape[0] / data.shape[0]

0.8003120124804992

In [18]:
# top 2 accuracy
data["exists"] = data.drop(data.columns[[0,1,2,3,6]], 1).isin(data["target"]).any(1)
sum(data['exists'])/ data.shape[0]

0.890795631825273

In [16]:
# top 3 accuracy
data["exists"] = data.drop(data.columns[[0,1,2,3]], 1).isin(data["target"]).any(1)
sum(data['exists'])/ data.shape[0]

0.9235569422776911

### load nlp model and get stop word list

In [8]:
# load spacy model 
import spacy
nlp = spacy.load("en_core_web_sm")

from spacy.lang.en.stop_words import STOP_WORDS
stop_words = list(STOP_WORDS)

### Get keyword list

In [9]:
keywords = []
for intent in list(set(data['Intent'])):
    keywords.extend(intent.strip().split(' '))
keyword_list = list(set(keywords))
keyword_list = [i.lower() for i in keyword_list if i.lower() not in stop_words]
keyword_list.append('nsip')

keyword_list_lemma = []
text = nlp(' '.join([w for w in keyword_list]))
for token in text:
    keyword_list_lemma.append(token.lemma_)
print(keyword_list_lemma)

['interest', 'statement', 'apply', 'home', 'tuition', 'pay', 'issue', 'service', 'dispute', 'internet', 'unable', 'transfer', 'reprice', 'pin', 'home', 'enquiry', 'damage', 'card', 'service', 'rebate', 'singapore', 'rejection', 'officer', 'ocbc', 'compliment', 'update', 'use', 'atm', 'customer', 'open', 'study', 'credit', 'activation', 'sponsorship', 'application', 'malaysia', 'compromise', 'suspension', 'loan', 'passbook', 'request', 'telegraphic', 'enquiries', 'statement', 'investment', 'debit', 'dispute', 'statement', 'fee', 'cancellation', 'transfer', 'close', 'mobile', 'log', 'annual', 'billing', 'banking', 'repayment', 'waiver', 'cancel', 'reward', 'status', 'student', 'increase', 'contribution', 'application', 'speak', 'telegraphic', 'card', 'access', 'account', 'token', 'waiver', 'card', 'opening', 'file', 'code', 'foreigner', 'refund', 'education', 'supplementary', 'cancel', 'uplift', 'interest', 'replace', 'security', 'banking', 'decrease', 'lose', 'srs', 'savings', 'car', 'l

### Get Linguistic feature

In [10]:
data['lemma'] = data['Questions'].apply(lambda x:' '.join([token.lemma_ for token in nlp(x) if token.lemma_ not in stop_words]))
data['keyword'] = data['lemma'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.lemma_ in keyword_list_lemma])))

data['noun'] = data['Questions'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['NOUN','PROPN'] and token.lemma_ not in stop_words])))
data['verb'] = data['Questions'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['VERB'] and token.lemma_ not in stop_words])))
data['adj'] = data['Questions'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['ADJ'] and token.lemma_ not in stop_words])))

data['noun'] = data['noun'].apply(lambda x: ' '.join([w for w in x]))
data['verb'] = data['verb'].apply(lambda x: ' '.join([w for w in x]))
data['adj'] = data['adj'].apply(lambda x: ' '.join([w for w in x]))
data['keyword'] = data['keyword'].apply(lambda x: ' '.join([w for w in x]))

In [11]:
data[data.lemma == 'want open account']

,Unnamed: 0,Intent,Questions,target,clusters_1,clusters_2,clusters_3,lemma,keyword,noun,verb,adj
368,368,Open Account,want open account,47,51,19,49,want open account,account open,account,want,open


## K-fold Cross validation results

### tf-idf + linguistic + cluster

In [12]:
# combine model score
countvector_cols = ['lemma', 'keyword', 'noun', 'verb']
top_clusters_cols = ['clusters_1', 'clusters_2', 'clusters_3']

feature_cols = countvector_cols + top_clusters_cols

# StratifiedKFold coss validation 
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(data[feature_cols], data['target'])
print(skf)

cv_scores_top1 = []
cv_scores_top2 = []
cv_scores_top3 = []

final_result = pd.DataFrame()

for train_index, test_index in skf.split(data[feature_cols], data['target']):
    # get train, test data for each chunk 
    X_train, X_test = data.loc[train_index,feature_cols], data.loc[test_index,feature_cols]
    y_train, y_test = data.loc[train_index,'target'], data.loc[test_index,'target']
    
    v_lemma = TfidfVectorizer()
    x_train_lemma = v_lemma.fit_transform(X_train['lemma'])
    x_test_lemma = v_lemma.transform(X_test['lemma'])
    vocab_lemma = dict(v_lemma.vocabulary_)
    
    v_keyword = TfidfVectorizer()
    x_train_keyword = v_keyword.fit_transform(X_train['keyword'])
    x_test_keyword = v_keyword.transform(X_test['keyword'])
    vocab_keyword = dict(v_keyword.vocabulary_)
    
    v_noun = TfidfVectorizer()
    x_train_noun = v_noun.fit_transform(X_train['noun'])
    x_test_noun = v_noun.transform(X_test['noun'])
    vocab_noun = dict(v_noun.vocabulary_)
    
    v_verb = TfidfVectorizer()
    x_train_verb = v_verb.fit_transform(X_train['verb'])
    x_test_verb = v_verb.transform(X_test['verb'])
    vocab_verb = dict(v_verb.vocabulary_)
    
#     v_adj = TfidfVectorizer()
#     x_train_adj = v_adj.fit_transform(X_train['adj'])
#     x_test_adj = v_adj.transform(X_test['adj'])
#     vocab_adj = dict(v_adj.vocabulary_)

    # combine all features 
    x_train_combined = hstack((x_train_lemma,x_train_keyword,x_train_noun,x_train_verb,X_train[top_clusters_cols].values),format='csr')
    x_train_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()+top_clusters_cols
    
    x_test_combined = hstack((x_test_lemma,x_test_keyword,x_test_noun,x_test_verb,X_test[top_clusters_cols].values),format='csr')
    x_test_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()+top_clusters_cols
   
    x_train_combined = pd.DataFrame(x_train_combined.toarray())
    x_train_combined.columns = x_train_combined_columns
    
    x_test_combined = pd.DataFrame(x_test_combined.toarray())
    x_test_combined.columns = x_test_combined_columns
    
    # build classifier
    clf = RandomForestClassifier(max_depth=50, n_estimators=1000)
    clf.fit(x_train_combined, y_train)
    
    
    probs = clf.predict_proba(x_test_combined)
    best_3 = pd.DataFrame(np.argsort(probs, axis=1)[:,-3:],columns=['top3','top2','top1'])
    best_3['top1'] = clf.classes_[best_3['top1']]
    best_3['top2'] = clf.classes_[best_3['top2']]
    best_3['top3'] = clf.classes_[best_3['top3']]
    
    result = pd.concat([best_3.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True), X_test[feature_cols].reset_index(drop=True)], axis=1)
    final_result = pd.concat([final_result, result])
    
    score_1 = result[result['top1'] == result['target']].shape[0] / result.shape[0]
    score_2 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])].shape[0] / result.shape[0]
    score_3 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])| (result['top3'] == result['target'])].shape[0] / result.shape[0]
    cv_scores_top1.append(score_1)
    cv_scores_top2.append(score_2)
    cv_scores_top3.append(score_3)
    
print(np.mean(np.array((cv_scores_top1))), np.std(np.array((cv_scores_top1))))
print(np.mean(np.array((cv_scores_top2))), np.std(np.array((cv_scores_top2))))
print(np.mean(np.array((cv_scores_top3))), np.std(np.array((cv_scores_top3))))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


/home/ubuntu/anaconda3/envs/alert/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning:

The least populated class in y has only 4 members, which is less than n_splits=5.



0.7863008720930232 0.013990250295576646
0.8830062984496123 0.02030175344843629
0.928234011627907 0.018760349023333362


#### Error analysis

In [13]:
intent_cols = ['target','top1','top2','top3', 'clusters_1',
       'clusters_2', 'clusters_3']
for col in intent_cols:
    final_result[col] = final_result[col].apply(lambda x: [intent for intent, index in intent2index.items() if index == x])
    final_result[col] = final_result[col].apply(lambda x: x[0])

In [14]:
final_result[final_result['target'] != final_result['top1']]

,top3,top2,top1,target,lemma,keyword,noun,verb,clusters_1,clusters_2,clusters_3
25,Account Fraud,Cancel credit card transaction,Cancel Credit or Debit Card,Cancel credit card transaction,-pron- cancel transaction -pron- card,cancel card transaction,cancel card -pron- transaction,,Cancel credit card transaction,Card Cancellation,Cancel Credit or Debit Card
27,CRS Enquiries,Increase credit card limit,Give a compliment,Credit Refund,credit refund,refund credit,refund credit,,Credit Refund,Rebates,Paying a cancelled credit card
31,File a complaint,Give a compliment,Speak with a customer service officer,Account Fraud,-pron- scamme,,-pron- scamme,,Statement request,Cancel ATM Card,Cancel Credit or Debit Card
32,Account Fraud,Uplift suspension on accounts,Dormant Account Activation,Account Fraud,think scamme -pron- account,account,account -pron- scamme,think,Open NISP Account,Open Account,Uplift suspension on accounts
38,CRS Enquiries,Give a compliment,File a complaint,Give a compliment,staff patient,,staff,,Give a compliment,File a complaint,Speak with a customer service officer
...,...,...,...,...,...,...,...,...,...,...,...
100,Dormant Account Activation,How to close my account,Uplift suspension on accounts,Request for account fee waiver,minimum balance need -pron- account,account,account balance -pron-,need,Request for account fee waiver,Increase credit card limit,Uplift suspension on accounts
101,Dormant Account Activation,Open Account,Uplift suspension on accounts,Request for account fee waiver,account minimum balance,account,balance account,,Request for account fee waiver,Increase credit card limit,Decrease credit card limit
108,Increase credit card limit,Credit card application rejection,Cancel Credit or Debit Card,Apply for credit or debit cards,high -pron- income credit card,card credit,income card -pron- credit,,Increase credit card limit,Unsuccessful card transaction,Credit Refund
114,Debit card statement,Card statements,Credit card statement,Card dispute,extra charge -pron- credit card statement,statement card credit,credit charge card statement -pron-,,Credit card statement,Card statements,Card dispute


### tf-idf + linguistic

In [15]:
# combine model score
countvector_cols = ['lemma', 'keyword', 'noun', 'verb']
top_clusters_cols = ['clusters_1', 'clusters_2', 'clusters_3']

feature_cols = countvector_cols + top_clusters_cols

# StratifiedKFold coss validation 
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(data[feature_cols], data['target'])
print(skf)

cv_scores_top1 = []
cv_scores_top2 = []
cv_scores_top3 = []

final_result = pd.DataFrame()

for train_index, test_index in skf.split(data[feature_cols], data['target']):
    # get train, test data for each chunk 
    X_train, X_test = data.loc[train_index,feature_cols], data.loc[test_index,feature_cols]
    y_train, y_test = data.loc[train_index,'target'], data.loc[test_index,'target']
    
    v_lemma = TfidfVectorizer()
    x_train_lemma = v_lemma.fit_transform(X_train['lemma'])
    x_test_lemma = v_lemma.transform(X_test['lemma'])
    vocab_lemma = dict(v_lemma.vocabulary_)
    
    v_keyword = TfidfVectorizer()
    x_train_keyword = v_keyword.fit_transform(X_train['keyword'])
    x_test_keyword = v_keyword.transform(X_test['keyword'])
    vocab_keyword = dict(v_keyword.vocabulary_)
    
    v_noun = TfidfVectorizer()
    x_train_noun = v_noun.fit_transform(X_train['noun'])
    x_test_noun = v_noun.transform(X_test['noun'])
    vocab_noun = dict(v_noun.vocabulary_)
    
    v_verb = TfidfVectorizer()
    x_train_verb = v_verb.fit_transform(X_train['verb'])
    x_test_verb = v_verb.transform(X_test['verb'])
    vocab_verb = dict(v_verb.vocabulary_)

    # combine all features 
    x_train_combined = hstack((x_train_lemma,x_train_keyword,x_train_noun,x_train_verb),format='csr')
    x_train_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()
    
    x_test_combined = hstack((x_test_lemma,x_test_keyword,x_test_noun,x_test_verb),format='csr')
    x_test_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()
   
    
    x_train_combined = pd.DataFrame(x_train_combined.toarray())
    x_train_combined.columns = x_train_combined_columns
    
    x_test_combined = pd.DataFrame(x_test_combined.toarray())
    x_test_combined.columns = x_test_combined_columns
    
    # build classifier
    clf = RandomForestClassifier(max_depth=50, n_estimators=800)
    clf.fit(x_train_combined, y_train)
    
    
    probs = clf.predict_proba(x_test_combined)
    best_3 = pd.DataFrame(np.argsort(probs, axis=1)[:,-3:],columns=['top3','top2','top1'])
    best_3['top1'] = clf.classes_[best_3['top1']]
    best_3['top2'] = clf.classes_[best_3['top2']]
    best_3['top3'] = clf.classes_[best_3['top3']]
    
    result = pd.concat([best_3.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True), X_test[feature_cols].reset_index(drop=True)], axis=1)
    final_result = pd.concat([final_result, result])
    
    score_1 = result[result['top1'] == result['target']].shape[0] / result.shape[0]
    score_2 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])].shape[0] / result.shape[0]
    score_3 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])| (result['top3'] == result['target'])].shape[0] / result.shape[0]
    cv_scores_top1.append(score_1)
    cv_scores_top2.append(score_2)
    cv_scores_top3.append(score_3)
    
print(np.mean(np.array((cv_scores_top1))), np.std(np.array((cv_scores_top1))))
print(np.mean(np.array((cv_scores_top2))), np.std(np.array((cv_scores_top2))))
print(np.mean(np.array((cv_scores_top3))), np.std(np.array((cv_scores_top3))))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


/home/ubuntu/anaconda3/envs/alert/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning:

The least populated class in y has only 4 members, which is less than n_splits=5.



0.764389534883721 0.018545295601492245
0.8814559108527131 0.01859948532579604
0.9188832364341085 0.015270786365466856


## Leave one out cross validation result

### tf-idf + linguistic 

In [16]:
from sklearn.model_selection import LeaveOneOut
# combine model score
countvector_cols = ['lemma', 'keyword', 'noun', 'verb']
top_clusters_cols = ['clusters_1', 'clusters_2', 'clusters_3']

feature_cols = countvector_cols + top_clusters_cols

# LeaveOneOut coss validation 
loo = LeaveOneOut()
loo.get_n_splits(data[feature_cols], data['target'])

print(loo)

cv_scores_top1 = []
cv_scores_top2 = []
cv_scores_top3 = []

final_result = pd.DataFrame()

for train_index, test_index in loo.split(data[feature_cols], data['target']):
    # print("TEST:", test_index)
    # get train, test data for each chunk 
    X_train, X_test = data.loc[train_index,feature_cols], data.loc[test_index,feature_cols]
    y_train, y_test = data.loc[train_index,'target'], data.loc[test_index,'target']
    
    v_lemma = TfidfVectorizer()
    x_train_lemma = v_lemma.fit_transform(X_train['lemma'])
    x_test_lemma = v_lemma.transform(X_test['lemma'])
    vocab_lemma = dict(v_lemma.vocabulary_)
    
    v_keyword = TfidfVectorizer()
    x_train_keyword = v_keyword.fit_transform(X_train['keyword'])
    x_test_keyword = v_keyword.transform(X_test['keyword'])
    vocab_keyword = dict(v_keyword.vocabulary_)
    
    v_noun = TfidfVectorizer()
    x_train_noun = v_noun.fit_transform(X_train['noun'])
    x_test_noun = v_noun.transform(X_test['noun'])
    vocab_noun = dict(v_noun.vocabulary_)
    
    v_verb = TfidfVectorizer()
    x_train_verb = v_verb.fit_transform(X_train['verb'])
    x_test_verb = v_verb.transform(X_test['verb'])
    vocab_verb = dict(v_verb.vocabulary_)

    # combine all features 
    x_train_combined = hstack((x_train_lemma,x_train_keyword,x_train_noun,x_train_verb),format='csr')
    x_train_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()
    
    x_test_combined = hstack((x_test_lemma,x_test_keyword,x_test_noun,x_test_verb),format='csr')
    x_test_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()
   
    
    x_train_combined = pd.DataFrame(x_train_combined.toarray())
    x_train_combined.columns = x_train_combined_columns
    
    x_test_combined = pd.DataFrame(x_test_combined.toarray())
    x_test_combined.columns = x_test_combined_columns
    
    # build classifier
    clf = RandomForestClassifier(max_depth=50, n_estimators=1000)
    clf.fit(x_train_combined, y_train)
    
    
    probs = clf.predict_proba(x_test_combined)
    best_3 = pd.DataFrame(np.argsort(probs, axis=1)[:,-3:],columns=['top3','top2','top1'])
    best_3['top1'] = clf.classes_[best_3['top1']]
    best_3['top2'] = clf.classes_[best_3['top2']]
    best_3['top3'] = clf.classes_[best_3['top3']]
    
    result = pd.concat([best_3.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True), X_test[feature_cols].reset_index(drop=True)], axis=1)
    final_result = pd.concat([final_result, result])
    
    score_1 = result[result['top1'] == result['target']].shape[0] / result.shape[0]
    score_2 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])].shape[0] / result.shape[0]
    score_3 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])| (result['top3'] == result['target'])].shape[0] / result.shape[0]
    cv_scores_top1.append(score_1)
    cv_scores_top2.append(score_2)
    cv_scores_top3.append(score_3)
    
print(np.mean(np.array((cv_scores_top1))), np.std(np.array((cv_scores_top1))))
print(np.mean(np.array((cv_scores_top2))), np.std(np.array((cv_scores_top2))))
print(np.mean(np.array((cv_scores_top3))), np.std(np.array((cv_scores_top3))))

LeaveOneOut()
TEST: [0]
TEST: [1]
TEST: [2]
TEST: [3]
TEST: [4]
TEST: [5]
TEST: [6]
TEST: [7]
TEST: [8]
TEST: [9]
TEST: [10]
TEST: [11]
TEST: [12]
TEST: [13]
TEST: [14]
TEST: [15]
TEST: [16]
TEST: [17]
TEST: [18]
TEST: [19]
TEST: [20]
TEST: [21]
TEST: [22]
TEST: [23]
TEST: [24]
TEST: [25]
TEST: [26]
TEST: [27]
TEST: [28]
TEST: [29]
TEST: [30]
TEST: [31]
TEST: [32]
TEST: [33]
TEST: [34]
TEST: [35]
TEST: [36]
TEST: [37]
TEST: [38]
TEST: [39]
TEST: [40]
TEST: [41]
TEST: [42]
TEST: [43]
TEST: [44]
TEST: [45]
TEST: [46]
TEST: [47]
TEST: [48]
TEST: [49]
TEST: [50]
TEST: [51]
TEST: [52]
TEST: [53]
TEST: [54]
TEST: [55]
TEST: [56]
TEST: [57]
TEST: [58]
TEST: [59]
TEST: [60]
TEST: [61]
TEST: [62]
TEST: [63]
TEST: [64]
TEST: [65]
TEST: [66]
TEST: [67]
TEST: [68]
TEST: [69]
TEST: [70]
TEST: [71]
TEST: [72]
TEST: [73]
TEST: [74]
TEST: [75]
TEST: [76]
TEST: [77]
TEST: [78]
TEST: [79]
TEST: [80]
TEST: [81]
TEST: [82]
TEST: [83]
TEST: [84]
TEST: [85]
TEST: [86]
TEST: [87]
TEST: [88]
TEST: [89]
TEST: 

### tf_idf + linguistic + cluster

In [17]:
from sklearn.model_selection import LeaveOneOut
# combine model score
countvector_cols = ['lemma', 'keyword', 'noun', 'verb']
top_clusters_cols = ['clusters_1', 'clusters_2', 'clusters_3']

feature_cols = countvector_cols + top_clusters_cols

# LeaveOneOut coss validation 
loo = LeaveOneOut()
loo.get_n_splits(data[feature_cols], data['target'])

print(loo)

cv_scores_top1 = []
cv_scores_top2 = []
cv_scores_top3 = []

final_result = pd.DataFrame()

for train_index, test_index in loo.split(data[feature_cols], data['target']):
    print("TEST:", test_index)
    # get train, test data for each chunk 
    X_train, X_test = data.loc[train_index,feature_cols], data.loc[test_index,feature_cols]
    y_train, y_test = data.loc[train_index,'target'], data.loc[test_index,'target']
    
    v_lemma = TfidfVectorizer()
    x_train_lemma = v_lemma.fit_transform(X_train['lemma'])
    x_test_lemma = v_lemma.transform(X_test['lemma'])
    vocab_lemma = dict(v_lemma.vocabulary_)
    
    v_keyword = TfidfVectorizer()
    x_train_keyword = v_keyword.fit_transform(X_train['keyword'])
    x_test_keyword = v_keyword.transform(X_test['keyword'])
    vocab_keyword = dict(v_keyword.vocabulary_)
    
    v_noun = TfidfVectorizer()
    x_train_noun = v_noun.fit_transform(X_train['noun'])
    x_test_noun = v_noun.transform(X_test['noun'])
    vocab_noun = dict(v_noun.vocabulary_)
    
    v_verb = TfidfVectorizer()
    x_train_verb = v_verb.fit_transform(X_train['verb'])
    x_test_verb = v_verb.transform(X_test['verb'])
    vocab_verb = dict(v_verb.vocabulary_)

    # combine all features 
    x_train_combined = hstack((x_train_lemma,x_train_keyword,x_train_noun,x_train_verb,X_train[top_clusters_cols].values),format='csr')
    x_train_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()+top_clusters_cols
    
    x_test_combined = hstack((x_test_lemma,x_test_keyword,x_test_noun,x_test_verb,X_test[top_clusters_cols].values),format='csr')
    x_test_combined_columns= v_lemma.get_feature_names()+v_keyword.get_feature_names()+v_noun.get_feature_names()+v_verb.get_feature_names()+top_clusters_cols
   
    x_train_combined = pd.DataFrame(x_train_combined.toarray())
    x_train_combined.columns = x_train_combined_columns
    
    x_test_combined = pd.DataFrame(x_test_combined.toarray())
    x_test_combined.columns = x_test_combined_columns
    
    # build classifier
    clf = RandomForestClassifier(max_depth=50, n_estimators=1000)
    clf.fit(x_train_combined, y_train)
    
    
    probs = clf.predict_proba(x_test_combined)
    best_3 = pd.DataFrame(np.argsort(probs, axis=1)[:,-3:],columns=['top3','top2','top1'])
    best_3['top1'] = clf.classes_[best_3['top1']]
    best_3['top2'] = clf.classes_[best_3['top2']]
    best_3['top3'] = clf.classes_[best_3['top3']]
    
    result = pd.concat([best_3.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True), X_test[feature_cols].reset_index(drop=True)], axis=1)
    final_result = pd.concat([final_result, result])
    
    score_1 = result[result['top1'] == result['target']].shape[0] / result.shape[0]
    score_2 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])].shape[0] / result.shape[0]
    score_3 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])| (result['top3'] == result['target'])].shape[0] / result.shape[0]
    cv_scores_top1.append(score_1)
    cv_scores_top2.append(score_2)
    cv_scores_top3.append(score_3)
    
print(np.mean(np.array((cv_scores_top1))), np.std(np.array((cv_scores_top1))))
print(np.mean(np.array((cv_scores_top2))), np.std(np.array((cv_scores_top2))))
print(np.mean(np.array((cv_scores_top3))), np.std(np.array((cv_scores_top3))))

LeaveOneOut()
TEST: [0]
TEST: [1]
TEST: [2]
TEST: [3]
TEST: [4]
TEST: [5]
TEST: [6]
TEST: [7]
TEST: [8]
TEST: [9]
TEST: [10]
TEST: [11]
TEST: [12]
TEST: [13]
TEST: [14]
TEST: [15]
TEST: [16]
TEST: [17]
TEST: [18]
TEST: [19]
TEST: [20]
TEST: [21]
TEST: [22]
TEST: [23]
TEST: [24]
TEST: [25]
TEST: [26]
TEST: [27]
TEST: [28]
TEST: [29]
TEST: [30]
TEST: [31]
TEST: [32]
TEST: [33]
TEST: [34]
TEST: [35]
TEST: [36]
TEST: [37]
TEST: [38]
TEST: [39]
TEST: [40]
TEST: [41]
TEST: [42]
TEST: [43]
TEST: [44]
TEST: [45]
TEST: [46]
TEST: [47]
TEST: [48]
TEST: [49]
TEST: [50]
TEST: [51]
TEST: [52]
TEST: [53]
TEST: [54]
TEST: [55]
TEST: [56]
TEST: [57]
TEST: [58]
TEST: [59]
TEST: [60]
TEST: [61]
TEST: [62]
TEST: [63]
TEST: [64]
TEST: [65]
TEST: [66]
TEST: [67]
TEST: [68]
TEST: [69]
TEST: [70]
TEST: [71]
TEST: [72]
TEST: [73]
TEST: [74]
TEST: [75]
TEST: [76]
TEST: [77]
TEST: [78]
TEST: [79]
TEST: [80]
TEST: [81]
TEST: [82]
TEST: [83]
TEST: [84]
TEST: [85]
TEST: [86]
TEST: [87]
TEST: [88]
TEST: [89]
TEST: 

## Baseline model -  tf-idf

In [18]:
# combine model score
feature_cols = ['Questions']

# StratifiedKFold coss validation 
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(data[feature_cols], data['target'])
print(skf)
cv_scores = []
cv_scores_top1 = []
cv_scores_top2 = []
cv_scores_top3 = []

for train_index, test_index in skf.split(data[feature_cols], data['target']):
    # get train, test data for each chunk 
    X_train, X_test = data.loc[train_index,feature_cols], data.loc[test_index,feature_cols]
    y_train, y_test = data.loc[train_index,'target'], data.loc[test_index,'target']
    
    v = TfidfVectorizer(ngram_range=(1, 1))
    x_train = v.fit_transform(X_train['Questions'])
    x_test = v.transform(X_test['Questions'])
    
    x_train = pd.DataFrame(x_train.toarray())
    x_test = pd.DataFrame(x_test.toarray())
    
    x_train.columns = v.get_feature_names()
    x_test.columns = v.get_feature_names()
    
    # build classifier
    clf = RandomForestClassifier(max_depth=25, n_estimators=300)
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    cv_scores.append(score)
    
    probs = clf.predict_proba(x_test)
    best_3 = pd.DataFrame(np.argsort(probs, axis=1)[:,-3:],columns=['top3','top2','top1'])
    best_3['top1'] = clf.classes_[best_3['top1']]
    best_3['top2'] = clf.classes_[best_3['top2']]
    best_3['top3'] = clf.classes_[best_3['top3']]
    result = pd.concat([best_3.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True)], axis=1)
    score_1 = result[result['top1'] == result['target']].shape[0] / result.shape[0]
    score_2 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])].shape[0] / result.shape[0]
    score_3 = result[(result['top1'] == result['target']) | (result['top2'] == result['target'])| (result['top3'] == result['target'])].shape[0] / result.shape[0]
    cv_scores_top1.append(score_1)
    cv_scores_top2.append(score_2)
    cv_scores_top3.append(score_3)
    
    
print(np.mean(np.array((cv_scores_top1))), np.std(np.array((cv_scores_top1))))
print(np.mean(np.array((cv_scores_top2))), np.std(np.array((cv_scores_top2))))
print(np.mean(np.array((cv_scores_top3))), np.std(np.array((cv_scores_top3))))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


/home/ubuntu/anaconda3/envs/alert/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning:

The least populated class in y has only 4 members, which is less than n_splits=5.



0.7722141472868217 0.01956516439124754
0.8626937984496124 0.02307247153203572
0.9157340116279069 0.016775988754018624
